In [1]:
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession

config = ConfigProto()
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)

/home/trojan/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/trojan/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/trojan/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/trojan/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:519: 

In [2]:
import tensorflow as tf
from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
# more info on callbakcs: https://keras.io/callbacks/ model saver is cool too.
from keras.callbacks import TensorBoard

from tensorflow import keras
from keras.models import Sequential
from keras.layers import *
from keras.applications.vgg16 import VGG16
from keras.preprocessing import image
from keras.applications.vgg16 import preprocess_input

from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint

import pickle
import time


train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=False,
        validation_split=0.1)

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    directory=r'/home/trojan/Desktop/pattern recognition/PB2/Implementation/data/abcde',
    target_size=(224, 224),
    color_mode="rgb",
    batch_size=1,
    class_mode="binary",
    shuffle=True,
    seed=42,
    subset="training"
)

valid_generator = train_datagen.flow_from_directory(
    directory=r'/home/trojan/Desktop/pattern recognition/PB2/Implementation/data/abcde',
    target_size=(224, 224),
    color_mode="rgb",
    batch_size=1,
    class_mode="binary",
    shuffle=True,
    seed=42,
    subset="validation"
)

'''test_generator = test_datagen.flow_from_directory(
    directory=r"/home/trojan/Downloads/kagglecatsanddogs_3367a/test data",
    target_size=(224, 224),
    color_mode="rgb",
    batch_size=1,
    class_mode="binary",
    shuffle=False,
    seed=42
)'''

NAME = "Cats-vs-dogs-CNN-64*2-{}".format(int(time.time()))


model = Sequential()

#model.add(VGG16(include_top=False, weights='imagenet', input_shape = (224,224, 3)))

model.add(Conv2D(256, (3, 3), padding='same', input_shape = (224, 224, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(256, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())  # this converts our 3D feature maps to 1D feature vectors

model.add(Dense(256))
model.add(Activation('relu'))

model.add(Dense(5))
model.add(Activation('softmax'))

tensorboard = TensorBoard(log_dir="logs/{}".format(NAME))

model.compile(loss='sparse_categorical_crossentropy',
              optimizer='sgd',
              metrics=['accuracy'],
              )


STEP_SIZE_TRAIN=train_generator.n//train_generator.batch_size
STEP_SIZE_VALID=valid_generator.n//valid_generator.batch_size

filepath="weights-improvement-{epoch:02d}-{val_accuracy:.2f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')
callbacks_list = checkpoint

model.fit_generator(generator=train_generator,
                    steps_per_epoch=STEP_SIZE_TRAIN,
                    validation_data=valid_generator,
                    validation_steps=STEP_SIZE_VALID,
                    epochs=10,
                    callbacks=[callbacks_list, tensorboard])
                    


Using TensorFlow backend.


Found 4182 images belonging to 5 classes.
Found 461 images belonging to 5 classes.




Epoch 1/10
4182/4182 [==============================] - 286s 68ms/step - loss: 1.6171 - accuracy: 0.1961 - val_loss: 1.6266 - val_accuracy: 0.1996

Epoch 00001: val_accuracy improved from -inf to 0.19957, saving model to weights-improvement-01-0.20.hdf5

Epoch 2/10
4182/4182 [==============================] - 294s 70ms/step - loss: 1.5786 - accuracy: 0.2523 - val_loss: 1.6534 - val_accuracy: 0.2798

Epoch 00002: val_accuracy improved from 0.19957 to 0.27983, saving model to weights-improvement-02-0.28.hdf5
Epoch 3/10
 524/4182 [==>...........................] - ETA: 4:13 - loss: 1.5163 - accuracy: 0.3149

KeyboardInterrupt: 

In [ ]:
model.evaluate_generator(generator=valid_generator,
steps=STEP_SIZE_VALID)

In [ ]:
STEP_SIZE_TEST=test_generator.n//test_generator.batch_size
test_generator.reset()
pred=model.predict_generator(test_generator,
steps=STEP_SIZE_TEST,
verbose=1)

In [ ]:
import numpy as np
predicted_class_indices=np.argmax(pred,axis=1)


In [ ]:
import pandas as pd
# Get classes by np.round
cl = np.round(pred)
# Get filenames (set shuffle=false in generator is important)
filenames=test_generator.filenames


# Data frame
results=pd.DataFrame({"file":filenames,"pr":pred[:,0], "class":cl[:,0]})

results.to_csv("results4.csv",index=False)

